Usage of the package
========

First, some imports

In [9]:
import gzip
import pickle
import random
from collections import namedtuple

from territories import Territory, MissingTreeCache

## Creation of the tree

The first step is to create a tree of known entities. This can be a very compute intensive task, depending on the tree size. That is why, by default, once created, the tree is stored on disk.

Here we will create a very simple tree out of the **tree.txt** file.

In [10]:
Node = namedtuple('Node', ('id', 'parent_id', 'label', 'level'))
split = lambda x: (arg if arg != 'null' else None for arg in x[:-1].split('; '))

try:
    Territory.load_tree()
except MissingTreeCache:
    with open("tree.txt", "r") as file:
        lines = file.readlines()
        stream = ([Node(*split(x) )for x in lines])
        Territory.build_tree(data_stream=stream, save_tree=False)

Then, you can start to create territories from arbitrary territoiral units.

Entities associated territories are represented in an efficient way : if all leaves of a parent node are included in the territory, they are simply replaced by their parent node.

In [11]:
# some node of the tree
print('\n'.join([f"{e.name} | {e.partition_type}" for e in random.sample(Territory.tree.nodes(), 12)]))

Bois-Guillaume | COM
Roquepine | COM
Le Boisle | COM
Moûtiers | COM
Campagnan | COM
Couin | COM
Oberhausbergen | COM
La Trinité-de-Thouberville | COM
Bellenod-sur-Seine | COM
Jouancy | COM
Telgruc-sur-Mer | COM
Le Ménil-Bérard | COM


In [12]:
territory = Territory.from_name()

In [13]:
Territory.tree.nodes()

[Marseille,
 Belley,
 Belleydoux,
 Valromey-sur-Séran,
 Bénonces,
 Bény,
 Béréziat,
 Bettant,
 Bey,
 Beynost,
 Billiat,
 Birieux,
 Biziat,
 Blyes,
 La Boisse,
 Boissey,
 Bolozon,
 Bouligneux,
 Bourg-en-Bresse,
 Bourg-Saint-Christophe,
 Boyeux-Saint-Jérôme,
 Boz,
 Brégnier-Cordon,
 Brénod,
 Brens,
 Bressolles,
 Brion,
 Briord,
 Buellas,
 La Burbanche,
 Ceignes,
 Cerdon,
 Certines,
 Cessy,
 Ceyzériat,
 Ceyzérieu,
 Chalamont,
 Chaleins,
 Chaley,
 Challes-la-Montagne,
 Challex,
 Champagne-en-Valromey,
 Champdor-Corcelles,
 Champfromier,
 Chanay,
 Chaneins,
 Chanoz-Châtenay,
 La Chapelle-du-Châtelard,
 Charix,
 Charnoz-sur-Ain,
 Château-Gaillard,
 Châtenay,
 Châtillon-la-Palud,
 Châtillon-sur-Chalaronne,
 Chavannes-sur-Reyssouze,
 Nivigne et Suran,
 Chaveyriat,
 Chazey-Bons,
 Chazey-sur-Ain,
 Cheignieu-la-Balme,
 Chevillard,
 Chevroux,
 Chevry,
 Chézery-Forens,
 Civrieux,
 Cize,
 Cleyzieu,
 Coligny,
 Collonges,
 Colomieu,
 Conand,
 Condamine,
 Condeissiat,
 Confort,
 Confrançon,
 Contrevoz,

In [ ]:
with open("tree_large.gzip", "rb") as file:
    lines = pickle.loads(gzip.decompress(file.read()))

stream = ([Node(*split(x) )for x in lines])
Territory.build_tree(data_stream=stream, save_tree=False)

a = Territory.from_name("COM:", "COM:", "DEP:")
b = Territory.from_name("COM:", "COM:", "DEP:")
c = Territory.from_name("COM:", "COM:", "DEP:")
d = Territory.from_name("COM:", "COM:", "DEP:")
e = Territory.from_name("COM:", "COM:", "DEP:")
f = Territory.from_name("COM:", "COM:", "DEP:")

## Operations on entities

Usual operation on territories works as expected :

In [6]:
# addition

print(a, c)
print(a + c) # This simplify to France

Marseille|Saint Etienne Grand Lyon|Île-de-France
France


In [7]:
# substraction

print(a, d)
print(a - d) # only Saint Etienne remains

Marseille|Saint Etienne Marseille|Grand Lyon
Saint Etienne


More importantly, sets operations are also supported

In [8]:
# intersection
print(f"Intersection of {a} and {d} is {a & d}")

# union
print(f"Union of {c} and {f} is {f | c}")

Intersection of Marseille|Saint Etienne and Marseille|Grand Lyon is Marseille
Union of Grand Lyon|Île-de-France and Marseille|Grand Lyon|Île-de-France is Marseille|Grand Lyon|Île-de-France


Territorial units may have parents or children, but Territory do not. As a territory may be formed of several territorial units, it has a LCA, a Lowest Common Ancestor.

In [ ]:
parent = territory.lowest_common_ancestor()